In [ ]:
def get_iou(a, b, epsilon=1e-5):
    """ Given two boxes `a` and `b` defined as a list of four numbers:
            [x1,y1,x2,y2]
        where:
            x1,y1 represent the upper left corner
            x2,y2 represent the lower right corner
        It returns the Intersect of Union score for these two boxes.

    Args:
        a:          (list of 4 numbers) [x1,y1,x2,y2]
        b:          (list of 4 numbers) [x1,y1,x2,y2]
        epsilon:    (float) Small value to prevent division by zero

    Returns:
        (float) The Intersect of Union score.
    """
    # COORDINATES OF THE INTERSECTION BOX
    x1 = max(a[0], b[0])
    y1 = max(a[1], b[1])
    x2 = min(a[2], b[2])
    y2 = min(a[3], b[3])

    # AREA OF OVERLAP - Area where the boxes intersect
    width = (x2 - x1)
    height = (y2 - y1)
    # handle case where there is NO overlap
    if (width<0) or (height <0):
        return 0.0
    area_overlap = width * height

    # COMBINED AREA
    area_a = (a[2] - a[0]) * (a[3] - a[1])
    area_b = (b[2] - b[0]) * (b[3] - b[1])
    area_combined = area_a + area_b - area_overlap

    # RATIO OF AREA OF OVERLAP OVER COMBINED AREA
    iou = area_overlap / (area_combined+epsilon)
    return iou

In [ ]:
import pandas as pd
import os
import torch
from torch.utils.data import DataLoader
import albumentations as A

from datasets.lungdatasets import SchenzenMontgomeryLungSegmentationDataset
from datasets.lungdatasets import CheXpertLungSegmentationDataset
from models.unet import ResNetUNet
from utils.utils import bce_dice_loss, dice_metric
import numpy as np
from tqdm import tqdm
import glob
import cv2

CHEXPERT_TRAIN = '../CheXpert-v1.0-small/train.csv'
BASE_MASKS = './intermediate/out_lung_mask/'
BASE_IMG = './data/chexpert-cardio-nofinding/'
BASE_EXTRA = 'CheXpert-v1.0-small/train/'

def get_transforms(size, test = True):
    #Do test-time augmentation?
    if test:
        return A.Compose([
        A.Resize(height=size, width=size, p=1.0)
        ])
    return A.Compose([
        A.Resize(height=size, width=size, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.3),
        A.Transpose(p=0.3),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.2, rotate_limit=45, p=0.3),
    ])

#Find Min skipping 0
def find_min(arr):
    min_val = 1000
    for idx, value in enumerate(arr):
        if value < min_val and value != 0:
            min_val = value
    return min_val

def find_chest_width_image(img,post_process=True):
    if post_process:
        img = post_process_image(img)
    start = np.argmax(img[:,:,1],axis=1)
    end = np.argmax(img[:,::-1,1],axis=1)
    h,w,c = img.shape
    return find_min(start), w - find_min(end), w

def find_chest_width(path,post_process=True):
    img = cv2.imread(path)
    if post_process:
        img = post_process_image(img)
    start = np.argmax(img[:,:,1],axis=1)
    end = np.argmax(img[:,::-1,1],axis=1)
    h,w,c = img.shape
    return find_min(start), w - find_min(end), w

def post_process_image(img,hull = True):
    
    dst = img[:,:,0]
    
    #kernel = np.ones((3, 3), np.uint8)
    #dst = cv2.erode(dst, kernel,iterations= 3) 

    contours, hierarchy = cv2.findContours(dst, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #create an empty image for contours
    img_contours = np.zeros(img.shape)
    # draw the contours on the empty image
    cs = [(c,cv2.contourArea(c)) for c in contours]
    cs.sort(key=lambda x:x[1])
    if hull:
        hulls = [cv2.convexHull(p[0]) for p in cs[-2:]]
        cv2.drawContours(img_contours, hulls, -1, (0,255,0), -1)
    else:
        contours2 = [p[0] for p in cs[-2:]]
        cv2.drawContours(img_contours, contours2, -1, (0,255,0), -1)
    return img_contours

def find_img(path):
    img = cv2.imread(path)
    return img.shape

import numpy as np
import cv2
from matplotlib.pyplot import imshow

ds_train_no_finding = CheXpertLungSegmentationDataset("./data/hand-label/nofinding.json", '../CheXpert-v1.0-small/train/'
                                                      , aug_transform=get_transforms(320)
                                                     , test = True)
ds_train_cardiomegaly = CheXpertLungSegmentationDataset("./data/hand-label/cardiomegaly-certain.json", '../CheXpert-v1.0-small/train/'
                                                        , aug_transform=get_transforms(320)
                                                       , test = True)

full_ds_chexpert1 = torch.utils.data.ConcatDataset([ds_train_no_finding, ds_train_cardiomegaly])
train_ds_chexpert1,val_ds_chexpert1 = torch.utils.data.random_split(full_ds_chexpert1, [len(full_ds_chexpert1) - 100, 100], generator=torch.Generator().manual_seed(42))
sample = np.uint8(full_ds_chexpert1[0][1].cpu().numpy() * 255)

ground_truth_lungs = {}
for img,mask,path in val_ds_chexpert1:
    print(path)
    mask = np.uint8(mask *255)
    ground_truth_lungs[path] = find_chest_width_image((np.stack([mask,mask,mask])*255).transpose(1,2,0))

In [ ]:
import os
import glob
import torch
from tqdm import tqdm
import cv2
import albumentations as A
import numpy as np
from datasets.lungdatasets import MEAN,STD
from models.unet import ResNetUNet

# 
IMAGE_SIZE = 512

LUNG_MODEL_WEIGHTS = './intermediate/lung_mask_weights'
PATH = "./intermediate/out_lung_mask3/"


base_path = 'C:/Users/ignacio/workspace/stanford/cs230/CheXpert-v1.0-small/train/'
CHEXPERT_VALIDATION_BASE = './data/chexpert-cardio-nofinding'

paths = os.listdir(CHEXPERT_VALIDATION_BASE)
inference_transforms = A.Compose([A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE, p=1.0)])

def load_image(base_path, path):
    path = path.replace('_','/',2)
    img_path = base_path + path
    image = cv2.imread(img_path,0)
    image = cv2.merge([image,image,image])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    augmented = inference_transforms(image=image)
    image = augmented['image']
    image = A.Normalize(mean=MEAN, std=STD)(image=image)["image"]
    return torch.FloatTensor(image).unsqueeze(0)

model = ResNetUNet().cuda()

#best_weights = sorted(glob.glob(LUNG_MODEL_WEIGHTS + "/*"), key=lambda x: x[8:-5])[-1]

#checkpoint = torch.load('./intermediate/lung_mask_weights/pretraining0.903464_.pth')
checkpoint = torch.load('./intermediate/lung_mask_weights/afterpretraining0.941366_.pth')
#checkpoint = torch.load('./intermediate/lung_mask_weights/nopretraining0.914747_.pth')
model.load_state_dict(checkpoint['state_dict'])

model.eval()
predictions_lungs = {}

for p in tqdm(paths): 
    if p in ground_truth_lungs:
        img = load_image(base_path, p)
        data_batch = img.permute(0, 3, 1, 2).cuda()
        outputs = model(data_batch)

        out_cut = np.copy(outputs.data.cpu().numpy())
        out_cut[np.nonzero(out_cut < 0.5)] = 0.0
        out_cut[np.nonzero(out_cut >= 0.5)] = 1.0

        mask = ((out_cut[0].transpose(1, 2, 0) * 255).astype(np.uint8))[:,:,0]
        prediction = find_chest_width_image((np.stack([mask,mask,mask]).transpose(1,2,0)))
        predictions_lungs[p] = prediction
        #cv2.imwrite(PATH + p, (out_cut[0].transpose(1, 2, 0) * 255).astype(np.uint8))
    
    #print(np.stack([mask,mask,mask]).shape)


# Heart

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import os
import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from datasets.heartdatasets import VinBigDataHeartDataset

import torchvision.transforms as T
from models.fastrcnn import get_model

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

def get_vinbigdata_dataframe(path):
    IMG_SIZE = 512
    df = pd.read_csv(path)
    df['x_min'] = IMG_SIZE * df['x_min'] / df['width']
    df['x_max'] = IMG_SIZE * df['x_max'] / df['width']
    df['y_min'] = IMG_SIZE * df['y_min'] / df['height']
    df['y_max'] = IMG_SIZE * df['y_max'] / df['height']
    df = df[df.class_name.eq('Cardiomegaly')]
    return df

from sklearn.model_selection import train_test_split

df = get_vinbigdata_dataframe('./data/vinbigdata/train.csv')
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)


ds_train = VinBigDataHeartDataset(df, './data/vinbigdata/train/', get_transform(train=False))
ds_test = VinBigDataHeartDataset(df, './data/vinbigdata/train/', get_transform(train=False))

from datasets.heartdatasets import CheXpertHeartDataset
import torch

ds_train_no_finding = CheXpertHeartDataset('./data/hand-label/nofinding/','../CheXpert-v1.0-small/train' ,get_transform(train=False), test= True)
ds_train_cardiomegaly = CheXpertHeartDataset('./data/hand-label/cardiomegaly-certain/','../CheXpert-v1.0-small/train' ,get_transform(train=False), test= True)

full_ds_chexpert2 = torch.utils.data.ConcatDataset([ds_train_no_finding, ds_train_cardiomegaly])

train_ds_chexpert2,val_ds_chexpert2 = torch.utils.data.random_split(full_ds_chexpert2, [len(full_ds_chexpert2) - 100, 100], generator=torch.Generator().manual_seed(42))

len(ds_train_no_finding), len(ds_train_cardiomegaly), len(full_ds_chexpert2)

print(len(val_ds_chexpert2))

ground_truth_heart = {}
for idx in range(len(val_ds_chexpert1)):
    image, data = val_ds_chexpert2[idx]
    if 'boxes' in data:
        ground_truth_heart[data['extra']] =  data['boxes'][0].cpu().numpy()
        #x0,y0,x1,y1
    print(data)

In [ ]:
train_ds_chexpert1

In [ ]:
import cv2
import torch
import os
import cv2
import torch
import torchvision.transforms as T
from PIL import Image
from models.fastrcnn import get_model

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

def evaluate_image(model, path,device):
    img_loaded = cv2.imread(path, cv2.IMREAD_COLOR)
    img_loaded = get_transform(train=False)(img_loaded)
    model.eval()
    with torch.no_grad():
        prediction = model([img_loaded.to(device)])
        return img_loaded,prediction[0]

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2

# get the model using our helper function
model = get_model(num_classes)
# move model to the right device
model.to(device)

name = 'modelPRETRAINING9'
model_params = torch.load(f'./intermediate/heart_weights/{name}.pth')
model.load_state_dict(model_params)

import os
from tqdm import tqdm
import pickle

CHEXPERT_VALIDATION_BASE = './data/chexpert-cardio-nofinding'

paths = os.listdir(CHEXPERT_VALIDATION_BASE)

predictions_heart = {}
for p in tqdm(paths):
    if p.replace('_','/',2) in ground_truth_heart:
        prediction = evaluate_image(model, CHEXPERT_VALIDATION_BASE+'/'+p,device)
        if len(prediction[1]['boxes']) > 0:
            predictions_heart[p.replace('_','/',2)] = ((prediction[1]['boxes'])[0],prediction[0].shape)
    #predictions.append(prediction)

In [ ]:
#ground_truth_heart

In [ ]:
print(len(ground_truth_lungs))
print(len(ground_truth_heart))

In [ ]:
#for i in range(99):
#    print(val_ds_chexpert1[i][2],val_ds_chexpert2[i][1]['extra'])

In [ ]:
#val_ds_chexpert1[1],val_ds_chexpert2[1]

In [ ]:
#for k in zip(predictions_heart,predictions_lungs):
#    print(k)

In [ ]:
count = 0
errors = []
errors_heart = []
errors_lung = []
h = []
l = []
r = []
ious = []
for x in list(ground_truth_heart):
    a = ground_truth_lungs[x.replace('/','_',2)]
    b = ground_truth_heart[x] 
    c = predictions_lungs[x.replace('/','_',2)]
    d = predictions_heart[x]
    count +=1
    
    start_lung, finish_lung, width_true = a
    start_heart,y0,finish_heart,y1 = b
    
    start_lung_pred, finish_lung_pred, width_pred = c
    box, width_heart = d
    (start_heart_pred,y0_pred,finish_heart_pred,y1_pred)= box.cpu().numpy()
    
    true_lung_ratio = (finish_lung-start_lung)/width_true
    true_heart_ratio = (finish_heart-start_heart)/width_heart[2]
    true_ctr = true_heart_ratio / true_lung_ratio   
    
    h.append(true_heart_ratio)
    l.append(true_lung_ratio)
    r.append(true_ctr)

    
    
    pred_lung_ratio = (finish_lung_pred-start_lung_pred)/width_pred
    pred_heart_ratio = (finish_heart_pred-start_heart_pred)/width_heart[2]
    
    pred_ctr = pred_heart_ratio / pred_lung_ratio
    
    iou = get_iou([start_heart_pred,y0_pred,finish_heart_pred,y1_pred], [start_heart,y0,finish_heart,y1], epsilon=1e-5)
    ious.append(iou)
    errors.append(abs(true_ctr - pred_ctr))
    errors_lung.append(abs(true_lung_ratio - pred_lung_ratio))
    errors_heart.append(abs(true_heart_ratio - pred_heart_ratio))


errors = np.array(errors)
errors_heart = np.array(errors_heart)
errors_lung = np.array(errors_lung)
ious = np.array(ious)

In [ ]:
np.array(h).mean(), np.array(l).mean(), np.array(r).mean()

In [ ]:
print(np.sum(ious > 0.5))
print(np.sum(ious > 0.75))
print(np.sum(ious > 0.90))
print(np.sum(ious > 0.95))

In [ ]:
len(errors)

In [ ]:
print('RMSE:', np.sqrt((errors ** 2).sum()/len(errors)))
print('Min:', errors.min())
print('Max:', errors.max())
print('Mean:', errors.mean())
print('Median:', np.median(errors))
print('STD:', errors.std())

In [ ]:
print('RMSE:', np.sqrt((errors_heart ** 2).sum()/len(errors_heart)))
print('Min:', errors_heart.min())
print('Max:', errors_heart.max())
print('Mean:', errors_heart.mean())
print('Median:', np.median(errors_heart))
print('STD:', errors_heart.std())

In [ ]:
print('RMSE:', np.sqrt((errors_lung ** 2).sum()/len(errors_lung)))
print('Min:', errors_lung.min())
print('Max:', errors_lung.max())
print('Mean:', errors_lung.mean())
print('Median:', np.median(errors_lung))
print('STD:', errors_lung.std())

In [ ]:
print('RMSE:', np.sqrt((ious ** 2).sum()/len(ious)))
print('Min:', ious.min())
print('Max:', ious.max())
print('Mean:', ious.mean())
print('Median:', np.median(ious))
print('STD:', ious.std())